<a href="https://colab.research.google.com/github/theysee/sberbank_reboot/blob/master/Copy_of_%D0%A8%D0%BA%D0%BE%D0%BB%D0%B0_DA_%D0%9F%D0%BE%D1%82%D0%BE%D0%BA_3_%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B6_%D0%BA%D1%80%D1%83%D0%BF%D0%BD%D0%BE%D0%B9_%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9_%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D0%B8_%D0%A1%D0%B5%D1%80%D0%B3%D0%B5%D0%B9_%D0%90%D1%80%D1%82%D0%B5%D0%B5%D0%B2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Суть Проекта в том, чтобы проанализировать продажи в крупной строительной компании.
Входные данные в формах Excel. Формируются программой автоматически, а значит всегда постоянны. Некоторые столбцы для анализа продаж они не нужны, поэтому я буду их удалять, чтобы не мешались.

In [0]:
import numpy as np
import pandas as pd
import pymysql
from datetime import datetime, timedelta, date
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Не понял, как можно получить список файлов в Colab, поэтому сделаю руками.
!ls "/content/drive/My Drive/Эскроу"

Застройщик_110.xls  Застройщик_140.xls	Застройщик_146.xls  Застройщик_176.xls
Застройщик_120.xls  Застройщик_141.xls	Застройщик_170.xls  Застройщик_28.xls


In [0]:
def pars(data_read):
  contract = data_read.iloc[5,6]
  contract = contract.split(' ')
  contract = contract[-3:-2]
  contract = int(contract[0])
  data_read.drop(data_read.columns[[0, 4, 5, 6, 7, 8, 9, 10, 11]], axis='columns', inplace=True)
  data_read.drop([0,1,2,3,4,5,6,7], inplace=True)
  data_read.columns = ['Date_open_escrow', 'Depo_summ', 'Date_pay']
  data_read['Date_open_escrow'] =  pd.to_datetime(data_read['Date_open_escrow'], format='%Y-%m-%d')
  data_read['Date_pay'] =  pd.to_datetime(data_read['Date_pay'], format='%Y-%m-%d')
  data_read['Contract'] = contract
  return data_read

In [5]:
data_read = pd.read_excel('/content/drive/My Drive/Эскроу/Застройщик_110.xls')
df = pars(data_read)
data_read = pd.read_excel('/content/drive/My Drive/Эскроу/Застройщик_140.xls')
df = df.append(pars(data_read), ignore_index = True)
data_read = pd.read_excel('/content/drive/My Drive/Эскроу/Застройщик_146.xls')
df = df.append(pars(data_read), ignore_index = True)
data_read = pd.read_excel('/content/drive/My Drive/Эскроу/Застройщик_176.xls')
df = df.append(pars(data_read), ignore_index = True)
data_read = pd.read_excel('/content/drive/My Drive/Эскроу/Застройщик_120.xls')
df = df.append(pars(data_read), ignore_index = True)
data_read = pd.read_excel('/content/drive/My Drive/Эскроу/Застройщик_141.xls')
df = df.append(pars(data_read), ignore_index = True)
data_read = pd.read_excel('/content/drive/My Drive/Эскроу/Застройщик_170.xls')
df = df.append(pars(data_read), ignore_index = True)
data_read = pd.read_excel('/content/drive/My Drive/Эскроу/Застройщик_28.xls')
df = df.append(pars(data_read), ignore_index = True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1818 entries, 0 to 1845
Data columns (total 4 columns):
Date_open_escrow    1818 non-null datetime64[ns]
Depo_summ           1818 non-null object
Date_pay            1818 non-null datetime64[ns]
Contract            1818 non-null int64
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 71.0+ KB


In [6]:
# Не знаю, как отработают файлы, чтобы не рисковать загружу в SQL.
!pip install PyMySQL

In [0]:
DB_HOST = '89.223.95.235'
DB_USER = 'student_12'
DB_USER_PASSWORD = 'student_12'
DB_NAME = 'student_12'
conn = pymysql.connect(DB_HOST, DB_USER, DB_USER_PASSWORD, DB_NAME)

In [8]:
pd.read_sql_query("show tables", conn)

,Tables_in_student_12
0,sales


In [0]:
#query = "DELETE FROM Sales"
#conn.cursor().execute(query)
#conn.commit()

In [0]:
#query = "DROP TABLE IF EXISTS Sales"
#conn.cursor().execute(query)
#conn.commit()

In [12]:
# По идее, ключ не нужен, но как SQL без ключа.
query = """
  CREATE TABLE IF NOT EXISTS Sales (
    sales_id INT AUTO_INCREMENT PRIMARY KEY,
    Date_open_escrow DATETIME,
    Depo_summ NUMERIC,
    Date_pay DATETIME,
    contract INT
    )
""" 

conn.cursor().execute(query)
conn.commit()

/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1050, "Table 'sales' already exists")
  result = self._query(query)


In [13]:
queries = []

for index, row in df.iterrows():
    querie = f'''INSERT INTO Sales (Date_open_escrow, Depo_summ, Date_pay, contract) VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}')'''
    queries.append(querie)

for q in queries:
  conn.cursor().execute(q)
#conn.commit()

/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'Depo_summ' at row 1")
  result = self._query(query)


In [18]:
# Записей 1818, значит всё в порядке.
df = pd.read_sql_query('''SELECT * FROM Sales''', conn)
df

,sales_id,Date_open_escrow,Depo_summ,Date_pay,contract
0,1,2019-06-28 16:08:00,3048000.0,2019-09-30,110
1,2,2019-06-28 09:13:00,2048000.0,2019-08-31,110
2,3,2019-06-28 10:45:00,2448000.0,2019-08-31,110
3,4,2019-06-28 09:40:00,98000.0,2021-09-30,110
4,5,2019-06-28 16:47:00,3700000.0,2019-12-31,110
...,...,...,...,...,...
1813,1814,2020-01-30 12:28:25,3520000.0,2021-09-30,28
1814,1815,2020-02-04 09:19:23,5150000.0,2021-09-30,28
1815,1816,2020-01-30 11:23:49,3150000.0,2021-09-30,28
1816,1817,2020-01-31 11:07:27,4190000.0,2021-09-30,28


In [0]:
df